In [2]:
import pandas as pd
import torch
import json
import numpy as np
from torch.utils.data import random_split, DataLoader
from neural_test import train_model, test_model, CustomDataset, make_and_eval
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt

In [3]:
class DynamicNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_hidden_layers):
        super(DynamicNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_hidden_layers = num_hidden_layers

        # Create the first layer
        self.layers = [torch.nn.Linear(input_size, hidden_size), torch.nn.ReLU()]

        # Create the hidden layers
        for _ in range(num_hidden_layers):
            self.layers.append(torch.nn.Linear(hidden_size, hidden_size))
            self.layers.append(torch.nn.ReLU())

        # Create the output layer
        self.layers.append(torch.nn.Linear(hidden_size, output_size))

        # Combine all layers
        self.model = torch.nn.Sequential(*self.layers)

    def forward(self, x):
        out = self.model(x)
        if not self.training:
            out = torch.clamp(out, min=1, max=5)
        return out

In [4]:
device = "cpu"
# Load JSON data
with open('../processed_data/topic-sentiment-total.json') as file:
    json_data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame.from_dict(json_data, orient='index')

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert DataFrame to numpy array
data = df.to_numpy()
df.head()

,pos_food,pos_service,pos_location,pos_clean,pos_price,neg_food,neg_service,neg_location,neg_clean,neg_price,rating
0,20,7,2,1,0,7,1,0,0,1,4.3
1,119,37,38,3,11,31,2,0,0,2,4.6
2,228,85,59,4,53,100,42,19,1,23,4.2
3,22,10,2,2,2,4,4,0,1,2,4.2
4,60,32,37,0,7,32,12,8,0,14,4.1


In [14]:
input_dim = 10
output_dim = 1 
hidden_dim = 40
num_hidden_layers = 4
learningRate = .01
epochs = 150

lambda1 = 0.0000 # l1 regularization parameter (sum of weights)
lambda2 = 0.0000 # l2 regularization parameter (sum of square of weights)

model = DynamicNN(input_dim, hidden_dim, output_dim, num_hidden_layers).to(device)
# Split the dataset
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(42))

trained_model, _ = make_and_eval(model, np.array(train_data), np.array(test_data), learningRate, epochs, lambda1, lambda2)

Mean Squared Error: 0.053394146263599396
Root Mean Squared Error: 0.23107174038887024
R-squared: 0.6974221742854183


### Grid Search

In [12]:
from itertools import product

param_grid = [
    [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
    [4, 5, 6, 7, 8],
]

epochs = 20

params = list(product(*param_grid))
all_r2 = []

for param in params:
    print(param[0], param[1])
    model = DynamicNN(input_dim, param[0], output_dim, param[1]).to(device)
    # Split the dataset
    train_size = int(0.8 * len(data))
    test_size = len(data) - train_size
    train_data, test_data = random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(42))

    trained_model, evals = make_and_eval(model, np.array(train_data), np.array(test_data), learningRate, epochs, lambda1, lambda2)
    all_r2.append(evals[3])

print(params[all_r2.index(max(all_r2))])

25 4
Mean Squared Error: 0.08151242882013321
Root Mean Squared Error: 0.2855038046836853
R-squared: 0.5380794064795463
25 5
Mean Squared Error: 0.14376987516880035
Root Mean Squared Error: 0.3791699707508087
R-squared: 0.18527433644562874
25 6
Mean Squared Error: 0.1732800453901291
Root Mean Squared Error: 0.4162692129611969
R-squared: 0.018044034900162265
25 7
Mean Squared Error: 0.17946627736091614
Root Mean Squared Error: 0.4236345887184143
R-squared: -0.017012592784868552
25 8
Mean Squared Error: 0.17648762464523315
Root Mean Squared Error: 0.4201042950153351
R-squared: -0.0001329683410478122
26 4
Mean Squared Error: 0.06558939069509506
Root Mean Squared Error: 0.2561042606830597
R-squared: 0.6283132568683816
26 5
Mean Squared Error: 0.06954747438430786
Root Mean Squared Error: 0.26371854543685913
R-squared: 0.6058833364373559
26 6
Mean Squared Error: 0.15878531336784363
Root Mean Squared Error: 0.39847874641418457
R-squared: 0.10018377458606575
26 7
Mean Squared Error: 0.176447436

In [7]:
# torch.save(model.state_dict(), "../nn4_40.pt")

In [8]:
input_dim = 10
output_dim = 1 
hidden_dim = 28
num_hidden_layers = 4
learningRate = .01
epochs = 150

k_folds = 5
# Create KFold object
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

models: list[torch.nn.Module] = []
all_mse = []
all_rmse = []
all_r2 = []

for fold, (train_ids, test_ids) in enumerate(kf.split(data)):
    print(f"Fold {fold + 1}/{k_folds}")

    model = DynamicNN(input_dim, hidden_dim, output_dim, num_hidden_layers).to(device)
    trained_model, evals = make_and_eval(model, data[train_ids], data[test_ids], learningRate, epochs, lambda1, lambda2)

    models.append(trained_model)
    all_mse.append(evals[1])
    all_rmse.append(evals[2])
    all_r2.append(evals[3])

print(f'Average Mean Squared Error: {np.mean(all_mse)}')
print(f'Average Root Mean Squared Error: {np.mean(all_rmse)}')
print(f'Average R-squared: {np.mean(all_r2)}')


Fold 1/5
Mean Squared Error: 0.07653889805078506
Root Mean Squared Error: 0.2766566574573517
R-squared: 0.5618250971351224
Fold 2/5
Mean Squared Error: 0.06457594782114029
Root Mean Squared Error: 0.2541179656982422
R-squared: 0.591398000679613
Fold 3/5
Mean Squared Error: 0.0751524493098259
Root Mean Squared Error: 0.2741394639015198
R-squared: 0.5729254434005384
Fold 4/5
Mean Squared Error: 10.408732414245605
Root Mean Squared Error: 3.2262566089630127
R-squared: -51.68719645057077
Fold 5/5
Mean Squared Error: 0.1691330224275589
Root Mean Squared Error: 0.41125786304473877
R-squared: 0.047257383115909324
Average Mean Squared Error: 2.1588265895843506
Average Root Mean Squared Error: 0.8884857296943665
Average R-squared: -9.982758105247916
